<a href="https://colab.research.google.with/github/maurorisonho/fraud-detection-neuromorphic/blob/main/notebooks/01-stdp_example.ipynb" target="_parent"><img src="https://colab.research.google.with/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STDP Example: Biological Learning

**Description:** Interactive Tutorial about the biological learning mechanism STDP (Spike-Timing-Dependent Plasticity) used in neuromorphic neural networks. Demonstrates how neurons learn timeral correlations automatically.

**Author:** Mauro Risonho de Paula Assumpção.
**Creation Date:** December 5, 2025.
**License:** MIT License.
**Development:** Human + AI Assisted Development (Claude Sonnet 4.5, Gemini 3 Pro Preview).

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
# Install the Brian2 library if not yet installed
try:
    import brian2
except ImportError:
    !pip install brian2
    import brian2

# Specific imports from brian2 instead of wildcard
from brian2 import (
    ms, mV, Hz, second,
    NeuronGroup, Synapses, SpikeMonitor, StateMonitor,
    SpikeGeneratorGroup, Network,
    defaultclock, run, device, start_scope,
    clip, prefs
)

# Configure to use numpy (avoids C++ compilation errors if headers are missing)
prefs.codegen.target = "numpy"

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print("✓ Imports completed!")

# STDP: Spike-Timing-Dependent Plasticity

**Description:** Interactive Tutorial about the biological learning mechanism STDP (Spike-Timing-Dependent Plasticity) used in neuromorphic neural networks. Demonstrates how neurons learn timeral correlations automatically.

**Author:** Mauro Risonho de Paula Assumpção
**Creation Date:** December 5, 2025
**License:** MIT License
**Development:** Human + AI-Assisted Development (Claude Sonnet 4.5, Gemini 3 Pro Preview).

---

This notebook explores the biological learning mechanism **STDP** used in neuromorphic neural networks.

## What is STDP?

STDP (Spike-Timing-Dependent Plasticity) is an **unsupervised** learning rule inspired by biological neurons:

- **if the pre-synaptic neuron fires BEFORE the post-synaptic** → **Potentiation** (weight ↑)
- **if the pre-synaptic neuron fires AFTER the post-synaptic** → **Depression** (weight ↓)

This allows the network to learn **timeral causal relationships** without explicit labels.

## 1. Setup and Imports

## 1. Classic STDP Curve

Visualize how the change in weight depends on the timeral difference between spikes.

In [ ]:
# STDP Parameters
tau_pre = 20.0 # ms - pre-synaptic time constant
tau_post = 20.0 # ms - post-synaptic time constant
A_pre = 0.01 # Potentiation Amplitude
A_post = -0.012 # Depression Amplitude

# Delta t (time difference)
dt_range = np.linspace(-100, 100, 500) # ms

# Calculate weight change
def stdp_weight_change(dt, tau_pre, tau_post, A_pre, A_post):
    """
    Calculates weight change via STDP.
    dt = t_post - t_pre
    """
    if dt > 0: # Post after Pre → Potentiation
        return A_pre * np.exp(-dt / tau_pre)
    else: # Post before Pre → Depression
        return A_post * np.exp(dt / tau_post)

weight_changes = np.array([stdp_weight_change(dt, tau_pre, tau_post, A_pre, A_post)
                           for dt in dt_range])

# Plot STDP curve
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(dt_range, weight_changes, linewidth=3, color='purple')
ax.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
ax.axvline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)

# Annotate regions
ax.fill_between(dt_range[dt_range > 0], 0, weight_changes[dt_range > 0],
                alpha=0.2, color='green', label='Potentiation (LTP)')
ax.fill_between(dt_range[dt_range < 0], 0, weight_changes[dt_range < 0],
                alpha=0.2, color='red', label='Depression (LTD)')

ax.set_xlabel('Δt = t_post - t_pre (ms)', fontsize=12)
ax.set_ylabel('Weight Change (Δw)', fontsize=12)
ax.set_title('STDP Curve: Spike-Timing-Dependent Plasticity', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Add annotations
ax.annotate('Pre → Post\n(Causal)', xy=(20, 0.008), fontsize=10,
            ha='center', color='green', fontweight='bold')
ax.annotate('Post → Pre\n(Anti-causal)', xy=(-20, -0.009), fontsize=10,
            ha='center', color='red', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ Interpretation:")
print("  - Δt > 0: pre-synaptic neuron fires BEFORE → Potentiation (strengthens connection)")
print("  - Δt < 0: pre-synaptic neuron fires AFTER → Depression (weakens connection)")
print("  - Effect decays exponentially with |Δt|")

PT_## 2. Simulation STDP with Brian2

Simulate two neurons connected with STDP and observe weight evolution.

In [ ]:
start_scope()

# Simulation parameters
duration = 100*ms  # type: ignore[operator]
defaultclock.dt = 0.1*ms  # type: ignore[operator]

print("⚙️ Configuring STDP simulation...")
print(f"Duration: {duration}")
print(f"Time step: {defaultclock.dt}\n")

# LIF neurons
tau_m = 10*ms  # type: ignore[operator]
tau_syn = 5*ms  # type: ignore[operator] # Synaptic time constant
v_rest = -70*mV
v_thresh = -50*mV
v_reset = -70*mV

# Added synaptic decay (dI_syn/dt)
eqs_post = '''
dv/dt = (v_rest - v + I_syn) / tau_m : volt
dI_syn/dt = -I_syn / tau_syn : volt
'''

# Create neurons
neuron_pre = SpikeGeneratorGroup(1, [0], [10]*ms)  # type: ignore[operator]
neuron_post = NeuronGroup(1, eqs_post, threshold='v > v_thresh',
                          reset='v = v_reset', method='euler')
neuron_post.v = v_rest
neuron_post.I_syn = 0*mV

# STDP Parameters
tau_pre_stdp = 20*ms  # type: ignore[operator]
tau_post_stdp = 20*ms  # type: ignore[operator]
A_pre_stdp = 0.01
A_post_stdp = -0.012
w_max = 1.0
w_min = 0.0

synapse_model = '''
w : 1
dApre/dt = -Apre / tau_pre_stdp : 1 (event-driven)
dApost/dt = -Apost / tau_post_stdp : 1 (event-driven)
'''

# Increased synaptic gain to ensure firing (w * 60*mV)
on_pre_stdp = '''
I_syn_post += w * 60 * mV
Apre += A_pre_stdp
w = clip(w + Apost, w_min, w_max)
'''

on_post_stdp = '''
Apost += A_post_stdp
w = clip(w + Apre, w_min, w_max)
'''

synapse = Synapses(neuron_pre, neuron_post,
                   model=synapse_model,
                   on_pre=on_pre_stdp,
                   on_post=on_post_stdp,
                   method='euler')
synapse.connect(i=0, j=0)
synapse.w = 0.5 # weight Initial

# Monitores
mon_pre = SpikeMonitor(neuron_pre)
mon_post = SpikeMonitor(neuron_post)
mon_weight = StateMonitor(synapse, 'w', record=True)
mon_voltage = StateMonitor(neuron_post, 'v', record=True)

# Execute simulation
print("⏳ Executing Brian2 simulation...")
import time
start_time = time.time()

net = Network(neuron_pre, neuron_post, synapse, mon_pre, mon_post, mon_weight, mon_voltage)
net.run(duration)

sim_time = time.time() - start_time

print(f"✅ Simulation completed in {sim_time:.3f}s!")
print(f"\n Results:")
print(f" Pre-synaptic spikes: {len(mon_pre.t)}")
print(f" Spikes post-synaptics: {len(mon_post.t)}")
print(f" weight Initial: {0.5:.3f}")
print(f" weight Final: {mon_weight.w[0][-1]:.3f}")
print(f" Change: {(mon_weight.w[0][-1] - 0.5):.3f} ({(mon_weight.w[0][-1] - 0.5)/0.5*100:+.1f}%)")

In [ ]:
# Visualize results
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Plot 1: Spikes
if len(mon_pre.t) > 0:
    axes[0].eventplot([mon_pre.t/ms], lineoffsets=1, linelengths=0.8,
                      linewidths=2, colors='blue', label='pre-synaptic')
if len(mon_post.t) > 0:
    axes[0].eventplot([mon_post.t/ms], lineoffsets=0, linelengths=0.8,
                      linewidths=2, colors='red', label='post-synaptic')

axes[0].set_ylabel('neuron')
axes[0].set_yticks([0, 1])
axes[0].set_yticklabels(['Post', 'Pre'])
axes[0].set_title('Raster Plot: Pre and Post-Synaptic Spikes', fontsize=12, fontweight='bold')
axes[0].legend(loc='upper right')
axes[0].grid(True, alpha=0.3)

# Plot 2: Evolution of the Synaptic weight
axes[1].plot(mon_weight.t/ms, mon_weight.w[0], linewidth=2.5, color='purple')
axes[1].axhline(0.5, color='gray', linestyle='--', alpha=0.5, label='weight Initial')
axes[1].set_ylabel('weight Synaptic (w)')
axes[1].set_title('Evolution of the Synaptic weight with STDP', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot 3: Voltage of the neuron post-synaptic
axes[2].plot(mon_voltage.t/ms, mon_voltage.v[0]/mV, linewidth=1.5, color='green')
axes[2].axhline(-50, color='red', linestyle='--', alpha=0.7, label='Threshold')
axes[2].axhline(-70, color='gray', linestyle='--', alpha=0.5, label='Resting')
axes[2].set_xlabel('time (ms)')
axes[2].set_ylabel('Voltage (mV)')
axes[2].set_title('Potencial of Membrana post-synaptic', fontsize=12, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. STDP with Input Patterns

Demonstrate how STDP learns temporal correlations in repeated patterns.

In [ ]:
start_scope()

# Simulate multiple pre-synaptic neurons
n_pre = 5
n_post = 1
duration = 500*ms  # type: ignore[operator]
defaultclock.dt = 0.1*ms  # type: ignore[operator]

print("⚙️ Configuring simulation with multiple neurons...")
print(f"Pre-synaptic neurons: {n_pre}")
print(f"Duration: {duration}\n")

# Generate temporal pattern (some neurons fire in sequence)
spike_pattern = [
    [10, 110, 210, 310, 410], # neuron 0: regular spikes
    [15, 115, 215, 315, 415], # neuron 1: slightly delayed
    [20, 120, 220, 320, 420], # neuron 2: more delayed
    [100, 200, 300, 400], # neuron 3: sparse spikes
    [50, 150, 250, 350, 450] # neuron 4: different phase
]

indices = []
times = []
print("📊 Spike patterns:")
for neuron_idx, spike_times in enumerate(spike_pattern):
    print(f"  Neuron {neuron_idx}: {len(spike_times)} spikes")
    for t in spike_times:
        indices.append(neuron_idx)
        times.append(t)

neuron_pre = SpikeGeneratorGroup(n_pre, indices, times*ms)  # type: ignore[operator]

# neuron post-synaptic
neuron_post = NeuronGroup(n_post, eqs_post, threshold='v > v_thresh',
                          reset='v = v_reset', method='euler')
neuron_post.v = v_rest

# Synapses with STDP
synapse = Synapses(neuron_pre, neuron_post,
                   model=synapse_model,
                   on_pre=on_pre_stdp,
                   on_post=on_post_stdp,
                   method='euler')
synapse.connect() # Connect all
synapse.w = 'rand() * 0.3 + 0.2' # Random initial weights [0.2, 0.5]

# Monitores
mon_pre = SpikeMonitor(neuron_pre)
mon_post = SpikeMonitor(neuron_post)
mon_weight = StateMonitor(synapse, 'w', record=True)

# Save weights initial
initial_weights = np.array(synapse.w).copy()

# Execute
print("\n⏳ Executing simulation of temporal patterns...")
start_time = time.time()

net = Network(neuron_pre, neuron_post, synapse, mon_pre, mon_post, mon_weight)
net.run(duration)

sim_time = time.time() - start_time
final_weights = np.array(synapse.w).copy()

print(f"✅ Simulation completed in {sim_time:.3f}s!")
print(f"\n📊 Synaptic Weights Analysis:")
for i in range(n_pre):
    delta = final_weights[i] - initial_weights[i]
    percentage = (delta / initial_weights[i]) * 100 if initial_weights[i] > 0 else 0
    print(f"  Neuron {i}: {initial_weights[i]:.3f} → {final_weights[i]:.3f} (Δ = {delta:+.3f}, {percentage:+.1f}%)")

In [ ]:
# Visualize evolution of the weights
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Plot 1: Temporal evolution of weights
for i in range(n_pre):
 axes[0].plot(mon_weight.t/ms, mon_weight.w[i], label=f'Synapse {i}', linewidth=2)

axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Synaptic Weight')
axes[0].set_title('Temporal Evolution of Synaptic Weights with STDP', fontsize=12, fontweight='bold')
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)

# Plot 2: Comparison before/after
x_pos = np.arange(n_pre)
width = 0.35

axes[1].bar(x_pos - width/2, initial_weights, width, label='Initial', alpha=0.7, color='lightblue')
axes[1].bar(x_pos + width/2, final_weights, width, label='Final', alpha=0.7, color='darkblue')

axes[1].set_xlabel('Pre-synaptic Neuron')
axes[1].set_ylabel('Synaptic Weight')
axes[1].set_title('Comparison: Initial vs Final Weights', fontsize=12, fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n💡 Interpretation:")
print("  - Neurons that fire consistently BEFORE the post-synaptic neuron are reinforced")
print("  - Neurons with inconsistent timing have reduced weights")
print("  - The network learns temporal correlations automatically!")

## 4. Application to Fraud Detection

How does STDP help in fraud detection?

### Scenario 1: Normal Temporal Sequence

**Legitimate Transaction:**
1. Login to the app (t=0ms)
2. Navigation to balance (t=500ms)
3. Selection of known beneficiary (t=2000ms)
4. Payment confirmation (t=3000ms)

**STDP learns:**
- Expected causal sequence
- Normal temporal intervals
- Reinforces connections that represent legitimate behavior

### Scenario 2: Anomalous Sequence (Fraud)

**Fraudulent Transaction:**
1. Login to the app (t=0ms)
2. Immediate transfer without navigation (t=50ms)
3. High value to new beneficiary (t=100ms)
4. Inconsistent geographic location (t=150ms)

**STDP detects:**
- Anomalous temporal pattern
- Sequence not reinforced during training
- High activation of "fraud" neurons

### Advantages of STDP:

1. **Unsupervised learning**: No need for explicit labels initially
2. **Continuous adaptation**: Learns new fraud patterns automatically
3. **Temporal sensitivity**: Detects anomalies in event sequences
4. **Efficiency**: Local weight updates (without backpropagation)
5. **Biologically plausible**: Inspired by the human brain

## 5. Conclusions

### STDP in Fraud Detection

**Mechanism:**
- Learns temporal correlations between transaction features
- Reinforces frequent legitimate patterns
- Detects deviations in temporal sequences

**Practical Applications:**
1. **Behavior analysis**: Sequence of actions in mobile banking
2. **Velocity detection**: Impossible transactions (e.g., purchases in different cities within minutes)
3. **Usage patterns**: Schedules, frequency, typical values
4. **Suspicious navigation**: Atypical page sequences

**Comparison with Traditional Methods:**

| Feature | STDP/SNN | DNN/LSTM |
|----------------|----------|----------|
| Temporal processing | Native | Emulated |
| Supervision | No | Yes |
| Latency | Ultra-low (~ms) | High (~100ms) |
| Energy consumption | Very low | High |
| Online adaptation | Yes | Difficult |
| Specialized hardware | Yes (Loihi, TrueNorth) | GPU |

### Future Directions

- Dedicated neuromorphic chips (Intel Loihi 2, IBM NorthPole)
- STDP + Reward modulation (artificial dopamine)
- Federated learning with STDP
- Explainability: Visualize learned weights

---

**Author:** Mauro Risonho de Paula Assumpção
**Project:** Neuromorphic Computing for Banking Cybersecurity